In [1]:
import pandas as pd

instrument="ES", 
root_path = "market_data/"
usecols = ["datetime", "open", "high", "low", "close", "high_before_low", "volume"] 
path = root_path+f"ES_1m_2-10-2020-12-00PM_3-10-2025-12-00PM_preprocessed.csv"
df = pd.read_csv(
    filepath_or_buffer=path,
    usecols=usecols,
    delimiter=";",
)
df.head()

,close,high,low,open,datetime,volume,high_before_low
0,3327.25,3327.75,3326.75,3326.75,2020-02-10 11:00:00,407,False
1,3327.25,3327.25,3326.50,3327.25,2020-02-10 11:01:00,126,True
2,3327.00,3327.25,3326.50,3327.00,2020-02-10 11:02:00,204,True
3,3326.50,3327.25,3326.50,3327.00,2020-02-10 11:03:00,233,True
4,3326.75,3326.75,3326.00,3326.50,2020-02-10 11:04:00,521,False


In [ ]:
# Il doit contenir 'timestamp' + ['open', 'high', 'low', 'close']
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# S'assurer que les données sont triées
df.sort_index(inplace=True)

# Grouper par date (chaque jour séparé)
grouped = df.groupby(df.index.date)

resampled_list = []

for date, group in grouped:
    group.index = pd.to_datetime(group.index)  # retransformer l'index en datetime si nécessaire
    # Rééchantillonnage à la minute près pour ce jour-là
    resampled = group.resample('1min').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })
    # Ne garder que les minutes où il y avait des données originales
    resampled = resampled.loc[group.index.min():group.index.max()]
    resampled_list.append(resampled)

# Recombiner tous les jours
df_resampled = pd.concat(resampled_list)

# Optionnel : remplir les trous dans les données si besoin
# df_resampled.fillna(method='ffill', inplace=True)

# Reset index si tu veux repasser en dataframe "plat"
df_resampled.reset_index(inplace=True)

In [ ]:
df_resampled

,datetime,open,high,low,close
0,2020-02-10 11:00:00,3326.75,3327.75,3326.75,3327.25
1,2020-02-10 11:01:00,3327.25,3327.25,3326.50,3327.25
2,2020-02-10 11:02:00,3327.00,3327.25,3326.50,3327.00
3,2020-02-10 11:03:00,3327.00,3327.25,3326.50,3326.50
4,2020-02-10 11:04:00,3326.50,3326.75,3326.00,3326.75
...,...,...,...,...,...
1871920,2025-03-10 10:55:00,5711.25,5711.25,5708.75,5710.25
1871921,2025-03-10 10:56:00,5710.25,5711.50,5709.75,5711.50
1871922,2025-03-10 10:57:00,5711.50,5712.00,5709.50,5709.75
1871923,2025-03-10 10:58:00,5709.50,5709.75,5708.25,5708.75


In [10]:
import plotly.graph_objects as go
start_date = pd.to_datetime("2024-04-24 12:40:00") 	
end_date = pd.to_datetime("2024-04-24 14:00:00")
df_plot = df_resampled[(start_date<=df_resampled["datetime"]) & (df_resampled["datetime"]<=end_date)]

fig = go.Figure(data=[go.Candlestick(
    x=df_plot['datetime'],
    open=df_plot['open'],
    high=df_plot['high'],
    low=df_plot['low'],
    close=df_plot['close'],
    name='SP500',
)])
fig.write_html("original_price.html")
fig.show()

In [6]:
from processing_functions import create_df
start_date = "2023-03-24 12:00"
end_date = "2025-02-10 10:00"
df_main = create_df(timeFramesUsedInMinutes=["1","5", "15"], instrument="ES", 
                    start_date = start_date, end_date = end_date, putVolumeInData=False) 
df_main.head()

,datetime,open,high,low,close,high_before_low,tenkan,kijun,ssa,ssb,...,PP,R1,R2,R3,S1,S2,S3,Previous Close,TR,ATR
0,2023-03-27 08:15:00,4010.75,4010.75,4008.50,4009.75,False,4009.75,4005.625,4004.7500,4012.125,...,4019.25,4032.0,4040.0,4060.75,4011.25,3998.5,3977.75,4011.00,2.50,1.785714
1,2023-03-27 08:16:00,4010.00,4011.00,4008.75,4009.50,True,4010.00,4006.000,4003.6875,4012.125,...,4019.25,4032.0,4040.0,4060.75,4011.25,3998.5,3977.75,4009.75,2.25,1.714286
2,2023-03-27 08:17:00,4009.25,4010.00,4008.75,4010.00,False,4010.00,4006.000,4003.6875,4012.125,...,4019.25,4032.0,4040.0,4060.75,4011.25,3998.5,3977.75,4009.50,1.25,1.696429
3,2023-03-27 08:18:00,4009.75,4010.75,4009.50,4010.75,False,4010.00,4006.875,4003.4375,4012.125,...,4019.25,4032.0,4040.0,4060.75,4011.25,3998.5,3977.75,4010.00,1.25,1.660714
4,2023-03-27 08:19:00,4010.50,4010.75,4009.50,4010.00,True,4010.00,4007.000,4003.1875,4012.125,...,4019.25,4032.0,4040.0,4060.75,4011.25,3998.5,3977.75,4010.75,1.25,1.607143


In [2]:
from processing_functions import load_object

filename = 'ES_m1_position=1_2025-04-19_17-51-03.088870'
filepath = f"trade_datas/ES/2023-03-24_12-00_2025-02-10_10-00/{filename}.pkl"
trades_database = load_object(filepath)

selected_id = 10
df = trades_database[selected_id][0]

In [8]:
import pandas as pd
start_date="2024-04-01 07:00:00"
end_date="2024-04-25 19:00:00"
df = df[(pd.to_datetime(start_date)<=df["entry_date"]) & (df["entry_date"]<=pd.to_datetime(end_date))]
df_losing = df[df["profit_from_start(%)"] > 0]

In [9]:
df_losing.tail()

,entry_date,exit_date,entry_price,exit_price,position,profit_including_fees_from_start(%),profit_from_start(%)
634,2024-04-23 13:45:00,2024-04-23 14:57:00,5083.50,5105.107143,Position.LONG,3.317173,3.325893
635,2024-04-24 12:22:00,2024-04-24 13:52:00,5121.75,5122.500000,Position.LONG,0.066280,0.075000
637,2024-04-24 20:05:00,2024-04-24 22:00:00,5090.00,5073.000000,Position.SHORT,1.691280,1.700000
638,2024-04-25 07:57:00,2024-04-25 11:04:00,5072.25,5071.542857,Position.SHORT,0.061994,0.070714
639,2024-04-25 11:27:00,2024-04-25 12:30:00,5072.00,5065.250000,Position.SHORT,0.666280,0.675000


In [ ]:
import pandas as pd
import plotly.graph_objects as go
# 1. Exemple de DataFrames (tu peux adapter avec les tiens)
# df_prices : historique des prix du SP500 (avec une colonne 'date' et 'price')
# df_trades : dataframe contenant les trades

# Choisir un index de trade, par exemple le 5e
trade_index = 639
trade = df_losing.loc[trade_index]

entry_date = pd.to_datetime(trade['entry_date'])
exit_date = pd.to_datetime(trade['exit_date'])

# 2. Filtrer les prix autour de la période du trade
df_plot = df_main[(df_main['datetime'] >= entry_date - pd.Timedelta(minutes=15)) & (df_main['datetime'] <= exit_date + pd.Timedelta(minutes=15))]
# Créer le graphique en chandelier
fig = go.Figure(data=[go.Candlestick(
    x=df_plot['datetime'],
    open=df_plot['open'],
    high=df_plot['high'],
    low=df_plot['low'],
    close=df_plot['close'],
    name='SP500',
)])
# fig.update_xaxes(dtick=1)

# ➤ Lignes Ichimoku
fig.add_trace(go.Scatter(x=df_plot['datetime'], y=df_plot['tenkan'], mode='lines', name='Tenkan-sen', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=df_plot['datetime'], y=df_plot['kijun'], mode='lines', name='Kijun-sen', line=dict(color='green')))
fig.add_trace(go.Scatter(x=df_plot['datetime'], y=df_plot['chikou'], mode='lines', name='Chikou Span', line=dict(color='purple')))

# ➤ Nuage Ichimoku (Kumo)
# Span A
fig.add_trace(go.Scatter(
    x=df_plot['datetime'],
    y=df_plot['ssa'],
    mode='lines',
    name='Senkou Span A',
    line=dict(color='rgba(0,200,0,0.5)'),
    fill=None
))
# Span B avec remplissage
fig.add_trace(go.Scatter(
    x=df_plot['datetime'],
    y=df_plot['ssb'],
    mode='lines',
    name='Senkou Span B',
    line=dict(color='rgba(200,0,0,0.5)'),
    fill='tonexty',  # crée le nuage entre Span B et la courbe précédente (Span A)
    fillcolor='rgba(100,100,100,0.2)'  # couleur du nuage
))

# ➤ Points d'entrée / sortie
fig.add_trace(go.Scatter(
    x=[entry_date], y=[trade['entry_price']],
    mode='markers+text', marker=dict(color='green', size=10),
    name='Entrée', text=["Entrée"], textposition="top center"
))
fig.add_trace(go.Scatter(
    x=[exit_date], y=[trade['exit_price']],
    mode='markers+text', marker=dict(color='red', size=10),
    name='Sortie', text=["Sortie"], textposition="bottom center"
))

# ➤ Mise en forme
fig.update_layout(
    title=f"Trade #{trade_index} avec Ichimoku Kumo - {trade['position']} | Profit: {trade['profit_including_fees_from_start(%)']}%",
    xaxis_title="Date",
    yaxis_title="Prix",
    xaxis_rangeslider_visible=False,
    template="plotly_white"
)

fig.show()

In [8]:
fig.write_html("trade_ichimoku.html")